# Playback saved model

In [1]:
pip install simpy

In [2]:
# Import the library to mount Google Drive
from google.colab import drive
# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

# Verify by listing the files in the drive
!ls /content/drive/My\ Drive/

import sys
sys.path.append('/content/drive/MyDrive/IntelligenzaArtificiale/qambo-main')


Mounted at /content/drive
 0512110799-Napolitano-Margherita-Maria.pdf  'Copia di Use case model.drawio'
 09_category_partition_30_11_2022.gdoc	      delega-accettazione-eventuale-supplenza-al-ds.doc
 2020.06.10.20127183v2.full.pdf		      DiagrammaNavigazionale2.drawio
'ANALISI DEL DATASET.gdoc'		     'Documento senza titolo.gdoc'
 biometria				      IntelligenzaArtificiale
'Colab Notebooks'			     'L-31 INFORMATICA'


In [3]:
################################################################################
#                           1 Import packages                                  #
################################################################################

from amboworld.environment import Env

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim

# Use a double ended queue (deque) for memory
# When memory is full, this will replace the oldest value with the new one
from collections import deque

# Supress all warnings (e.g. deprecation warnings) for regular use
import warnings

warnings.filterwarnings("ignore")

In [4]:
MODEL_PATH = './models/'

MODELS = {
    'ddqn':'ddqn_policy.pt',
    '3dqn': '3dqn_policy.pt'
}

# Replicate runs
TEST_EPISODES = 30

# Set whether to display on screen (slows model)
DISPLAY_ON_SCREEN = False

# SIM PARAMETERS
RANDOM_SEED = 42
SIM_DURATION = 5000
NUMBER_AMBULANCES = 9
NUMBER_INCIDENT_POINTS = 3
INCIDENT_RADIUS = 2
NUMBER_DISPTACH_POINTS = 25
AMBOWORLD_SIZE = 50
INCIDENT_INTERVAL = 20
EPOCHS = 2
AMBO_SPEED = 60
AMBO_FREE_FROM_HOSPITAL = False

In [5]:
all_results = dict()

## Random action model

In [6]:
results = dict()
results['call_to_arrival'] = []
results['assign_to_arrival'] = []
results['demand_met'] = []

sim = Env(
        random_seed = RANDOM_SEED,
        duration_incidents = SIM_DURATION,
        number_ambulances = NUMBER_AMBULANCES,
        number_incident_points = NUMBER_INCIDENT_POINTS,
        incident_interval = INCIDENT_INTERVAL,
        number_epochs = EPOCHS,
        number_dispatch_points = NUMBER_DISPTACH_POINTS,
        incident_range = INCIDENT_RADIUS,
        max_size = AMBOWORLD_SIZE,
        ambo_kph = AMBO_SPEED,
        ambo_free_from_hospital = AMBO_FREE_FROM_HOSPITAL
    )

for run in range(TEST_EPISODES):

    # Reset game environment and get first state observations
    state = sim.reset()

    # Continue loop until episode complete
    while True:
        action = random.randint(0, sim.action_number -1)
        state_next, reward, terminal, info = sim.step(action)

        # Actions to take if end of game episode
        if terminal:

            print(f'Run: {run}, ', end='')
            mean_assignment_to_arrival = np.mean(info['assignment_to_arrival'])
            print(f'Mean assignment to arrival: {mean_assignment_to_arrival:4.1f}, ', end='')
            mean_call_to_arrival = np.mean(info['call_to_arrival'])
            print(f'Mean call to arrival: {mean_call_to_arrival:4.1f}, ', end='')
            demand_met = info['fraction_demand_met']
            print(f'Demand met {demand_met:0.3f}')

            results['call_to_arrival'].append(mean_call_to_arrival)
            results['assign_to_arrival'].append(mean_assignment_to_arrival)
            results['demand_met'].append(demand_met)

            break

results = pd.DataFrame(results)
filename = '/content/drive/MyDrive/IntelligenzaArtificiale/qambo-main/3Incidents/new_output/results_random_action.csv'
results.to_csv(filename, index=False)

Run: 0, Mean assignment to arrival: 18.6, Mean call to arrival: 19.1, Demand met 0.999
Run: 1, Mean assignment to arrival: 18.7, Mean call to arrival: 19.2, Demand met 1.000
Run: 2, Mean assignment to arrival: 18.8, Mean call to arrival: 19.3, Demand met 1.000
Run: 3, Mean assignment to arrival: 18.5, Mean call to arrival: 19.0, Demand met 0.999
Run: 4, Mean assignment to arrival: 18.8, Mean call to arrival: 19.3, Demand met 1.000
Run: 5, Mean assignment to arrival: 18.6, Mean call to arrival: 19.2, Demand met 0.999
Run: 6, Mean assignment to arrival: 18.9, Mean call to arrival: 19.4, Demand met 1.000
Run: 7, Mean assignment to arrival: 18.9, Mean call to arrival: 19.4, Demand met 1.000
Run: 8, Mean assignment to arrival: 18.8, Mean call to arrival: 19.3, Demand met 0.999
Run: 9, Mean assignment to arrival: 18.8, Mean call to arrival: 19.4, Demand met 1.000
Run: 10, Mean assignment to arrival: 18.7, Mean call to arrival: 19.2, Demand met 0.999
Run: 11, Mean assignment to arrival: 18.9,

In [7]:
results

,call_to_arrival,assign_to_arrival,demand_met
0,19.135641,18.608330,0.999
1,19.208865,18.678246,1.000
2,19.280453,18.764807,1.000
3,19.007780,18.479319,0.999
4,19.320563,18.795998,1.000
5,19.155511,18.604505,0.999
6,19.382568,18.865421,1.000
7,19.447848,18.928207,1.000
8,19.336307,18.814434,0.999
9,19.365920,18.847457,1.000


In [8]:
results.describe()

,call_to_arrival,assign_to_arrival,demand_met
count,30.000000,30.000000,30.000000
mean,19.248020,18.725304,0.999733
std,0.192675,0.197460,0.000450
min,18.782609,18.267781,0.999000
25%,19.135435,18.604789,0.999250
50%,19.282662,18.769025,1.000000
75%,19.388846,18.870901,1.000000
max,19.611666,19.111381,1.000000
